<div class="alert alert-info" role="alert">
  SIMPLE XGB KFOLD VALIDATION APPROACH
</div>



<center> <h1 style="background-color:purple; color:white" >A simple XGB KFOLD VALIDATION</h1>    
    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:purple; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#libraries" role="tab" aria-controls="profile" style="color:purple">Importing Libraries<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
  <a id="section2" class="list-group-item list-group-item-action" data-toggle="list" href="#load" role="tab" aria-controls="messages" style="color:purple">Importing Files<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#missing" role="tab" aria-controls="settings" style="color:purple">Reduce Memory Usage<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#summary" role="tab" aria-controls="settings" style="color:purple">CV<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#rfm" role="tab" aria-controls="settings" style="color:purple">Parameters<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">5</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#visualize" role="tab" aria-controls="settings" style="color:purple"> Train<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">6</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#notebooks" role="tab" aria-controls="settings" style="color:purple"> Submission<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">7</span></a>

# Importing Libraries

In [ ]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
from math import sqrt
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error,f1_score, accuracy_score

# Importing Files

In [ ]:
train = pd.read_csv('../input/ion-switch-model-ready-data-frame-to-work-locally/train_ion_switch.csv')
test  = pd.read_csv('../input/ion-switch-model-ready-data-frame-to-work-locally/test_ion_switch.csv')

# Reduce Memory Usage

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
y     = train['open_channels']
train = train.drop(['open_channels'],axis=1)

# CV

In [ ]:
id_train = train['time']
id_test  = test['time']

train = train.drop('time', axis = 1)
test  = test.drop( 'time', axis = 1)

n_folds = 3
folds = KFold(n_splits = 3, shuffle=True, random_state=25)

# Parameters

In [ ]:
params = {'min_child_weight': 5, 
          'colsample_bytree': 0.1, 
          'max_depth': 21,
          'subsample': 0.1, 
          'nthread': -1, 
          'booster' : 'gbtree', 
          'silent': 1, 
          'gamma' : 0,
          'eval_metric': 'rmse',
          'objective': 'reg:squarederror'}

# Train

In [ ]:
mvalid = np.zeros(len(train))
mfull  = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train.values)):
    print('----')
    print("fold n°{}".format(fold_))
    
    x0,y0 = train.iloc[trn_idx], y[trn_idx]
    x1,y1 = train.iloc[val_idx], y[val_idx]
    
    trn_data = xgb.DMatrix(x0, label= y0); val_data = xgb.DMatrix(x1, label= y1)
    
    num_round = 250
    clf = xgb.train(params, trn_data, num_round, evals = [(val_data, "val_data")], 
                    verbose_eval=25, early_stopping_rounds = 10)
    mvalid[val_idx] = clf.predict(xgb.DMatrix(x1), ntree_limit=clf.best_iteration)
    
    mfull += clf.predict(xgb.DMatrix(test), ntree_limit=clf.best_iteration) / folds.n_splits
    
np.sqrt(mean_squared_error(mvalid, y))

# Submission

In [ ]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv")

submission = pd.DataFrame()
submission['time']  = sub['time']
submission['open_channels'] = mfull
submission['open_channels'] = submission['open_channels'].round(decimals=0)
submission['open_channels'] = submission['open_channels'].astype(int)
submission.to_csv('submission.csv', float_format='%0.4f', index = False)

In [ ]:
submission.tail()

<div class="alert alert-info" role="alert">
  Thanks for checkıng it out!
</div>